In [1]:
### Installations

# !pip uninstall pandas
# !pip install pandas==1.1.5
# !pip install pyLDAvis
# !pip install pyLDAvis.gensim
# !pip install bokeh
# !pip install gensim
# !pip install spacy
# !pip install logging
# !pip install wordcloud
# !pip install nltk
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# !pip install -U seaborn
# !pip install translators --upgrade
# !pip install --upgrade pip

In [3]:
### Imports
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import requests
import xml
from requests.utils import requote_uri
import xml.etree.ElementTree as ET 

In [ ]:
def splice(doc):
    counts = 'numChars numCharsMinusSpacesAndPunctuation numWords numSentences numPunctuation'
    speech = 'numNouns nounRatio numVerbs verbRatio numAdjectives adjectiveRatio numAdverbs adverbRatio'
    immediacy = 'numPassiveVerbs passiveVerbRatio'
    pronouns = 'firstPersonSingular firstPersonSingularRatio firstPersonPlural firstPersonPluralRatio secondPerson secondPersonRatio thirdPersonSingular thirdPersonSingularRatio thirdPersonPlural thirdPersonPluralRatio'
    positive_self = 'iCanDoIt doKnow posSelfImage'
    negative_self = 'iCantDoIt dontKnow negSelfImage'
    influence = 'numImperatives suggestionPhrases inflexibility contradict totalDominance dominanceRatio numAgreement agreementRatio'
    deference = "askPermission seekGuidance totalSubmissiveness submissivenessRatio"
    affect = "Imagery Pleasantness Activation"
    complexity = "avgWordLength avgSentenceLength numSyllables avgSyllablesPerWord numWordsWith3OrMoreSyllables rateWordsWith3OrMoreSyllables numWordsWith6OrMoreChars rateWordsWith6OrMoreChars numWordsWith7OrMoreChars rateWordsWith7OrMoreChars LexicalDiversity complexityComposite"
    style = "hedgeVerb hedgeConj hedgeAdj hedgeModal hedgeAll numDisfluencies disfluencyRatio numInterjections interjectionRatio numSpeculate speculateRatio Expressivity numIgnorance ignoranceRatio Pausality questionCount questionRatio hedgeUncertain"
    tense = 'pastTense pastTenseRatio presentTense presentTenseRatio'
    sentiment = "SWNpositivity SWNnegativity SWNobjectivity"
    readability = "ARI FRE FKGL CLI LWRF FOG SMOG DALE LIX RIX FRY"

    url = f"http://splice.cmi.arizona.edu/SPLICE/{doc}/{counts} {speech} {immediacy} {pronouns} {influence} {deference} {affect} {complexity} {style} {tense} {sentiment} {readability}"
    url_encoded = requote_uri(url)
    req = requests.request("GET", url_encoded)

    results = []
    root = ET.fromstring(req.text)
    for child in list(root.getchildren()):
        results.append(list([child.tag, child.text]))

    return results
      

In [4]:
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

df = pd.read_csv("noReplies.csv")
df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...


,Unnamed: 0,id,handle,followers,text,lang,possibly_sensitive,retweet_count,reply_count,like_count,...,contains_quote,is_reply,num_referenced_tweets,url_image,num_hashtags,text_first_hashtag,num_mentions,num_cashtags,num_polls,interaction_score
0,0,1470814862871777287,Schlotzskys,19593,Challenging you to not bite your phone. https:...,en,False,2,5,29,...,False,False,0,NaN,0,NaN,0,0,0,0.001710
1,1,1470095650150928385,Schlotzskys,19593,VOILA—this secret Santa is bringing Schlotzsky...,en,False,2,0,1,...,False,False,0,NaN,0,NaN,0,0,0,0.000332
2,2,1469339638334697475,Schlotzskys,19593,"No, no, no...you stay where you are. We'll com...",en,False,1,5,15,...,False,False,0,NaN,0,NaN,0,0,0,0.000791
3,3,1469032106068254736,Schlotzskys,19593,Tag your better half. #SharingIsCaring https:/...,en,False,2,5,34,...,False,False,0,NaN,1,SharingIsCaring,0,0,0,0.001837
4,4,1467906260418895881,Schlotzskys,19593,No holiday spread is complete without Cinnabon...,en,False,2,0,4,...,False,False,0,NaN,0,NaN,0,0,0,0.000408


In [ ]:
df['sentiment_overall'] = 0.0
df['sentiment_neg'] = 0.0
df['sentiment_neu'] = 0.0
df['sentiment_pos'] = 0.0

for row in df.itertuples():
    sentiment = sia.polarity_scores(row[5])
    df.loc[row[0], 'sentiment_overall'] = sentiment['compound']
    df.loc[row[0], 'sentiment_neg'] = sentiment['neg']
    df.loc[row[0], 'sentiment_neu'] = sentiment['neu']
    df.loc[row[0], 'sentiment_pos'] = sentiment['pos']